In [1]:
import os
import mmcv
import torch
from mmcv import Config
from mmseg.datasets import build_dataloader, build_dataset
from mmseg.models import build_segmentor
from mmseg.apis import single_gpu_test
from mmcv.runner import load_checkpoint
from mmcv.parallel import MMDataParallel

import pandas as pd
import numpy as np
import json

In [2]:
classes = ('Background','General trash', 'Paper', 'Paper pack', 'Metal', 'Glass', 'Plastic', 'Styrofoam', 'Plastic bag', 'Battery', 'Clothing')
palette = [[0,0,0],[192,0,128],[0,128,192],[0,128,64],[128,0,0],[64,0,128],[64,0,192],[192,128,64],[192,192,128],[64,64,128],[128,0,192]]
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset
import os.path as osp


In [3]:
# epoch 지정해주기
epoch = 'last_50_only_tta'
# work dir 설정해주기
work_dir = '/opt/ml/mmsegmentation/_MyModel/submission'

# config file 들고오기
# config file은 수정해서 사용
cfg = Config.fromfile('/opt/ml/mmsegmentation/_MyModel/_models_/knet_s3_upernet_convnext-d8_8x2_512x512.py')
root='/opt/ml/input/mmseg/images'

# dataset config 수정
# cfg.data.test.img_dir = root# cfg.data.test.pipeline[1]['img_scale'] = (512,512)
cfg.data.test.test_mode = True
# cfg.data.test.img_dir = 'images'
# cfg.data.test.split = 'splits/test.txt'
cfg.data.samples_per_gpu = 1
cfg.work_dir = work_dir
# cfg.model.decode_head.num_classes = 11
# cfg.model.auxiliary_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.model.train_cfg = None

In [4]:
dataset = build_dataset(cfg.data.test)
data_loader = build_dataloader(
        dataset,
        samples_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)

2022-05-12 03:38:48,312 - mmseg - INFO - Loaded 624 images


In [5]:
# checkpoint path
checkpoint_path = '/opt/ml/mmsegmentation/_MyModel/work_dirs/knet_last/epoch_50_only.pth'

model = build_segmentor(cfg.model, test_cfg=cfg.get('test_cfg'))
checkpoint = load_checkpoint(model, checkpoint_path, map_location='cpu')
model.CLASSES = dataset.CLASSES
model = MMDataParallel(model.cuda(), device_ids=[0])

load checkpoint from local path: /opt/ml/mmsegmentation/_MyModel/work_dirs/knet_last/epoch_50_only.pth


In [6]:
output = single_gpu_test(model, data_loader)

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 624/624, 0.6 task/s, elapsed: 961s, ETA:     0s

In [7]:
# submission 양식에 맞게 output 후처리

input_size = 512
output_size = 256

submission = pd.read_csv("/opt/ml/mmsegmentation/_MyModel/submission/sample_submission.csv", index_col=None)
json_dir = os.path.join("/opt/ml/input/data/test.json")

submission_check = pd.read_csv("/opt/ml/mmsegmentation/_MyModel/submission/sample_submission.csv", index_col=None)
check_dir = '/opt/ml/input/mmseg/test_check'
os.makedirs(check_dir+'/csv',exist_ok=True)
os.makedirs(check_dir+'/image',exist_ok=True)


with open(json_dir, "r", encoding="utf8") as outfile:
    datas = json.load(outfile)
import matplotlib.pyplot as plt
from PIL import Image
# PredictionString 대입
palette = [[0,0,0],[192,0,128],[0,128,192],[0,128,64],[128,0,0],[64,0,128],[64,0,192],[192,128,64],[192,192,128],[64,64,128],[128,0,192]]

for image_id, predict in enumerate(output):

    image_id = datas["images"][image_id]
    file_name = image_id["file_name"]
    
    temp_mask = []
    predict = predict.reshape(1, 512, 512)

    mask = predict.reshape((1, output_size, input_size//output_size, output_size, input_size//output_size)).max(4).max(2) # resize to 256*256
    temp_mask.append(mask)
    oms = np.array(temp_mask)
    oms = oms.reshape([oms.shape[0], output_size*output_size]).astype(int)
    string = oms.flatten()

    submission = pd.concat([submission, pd.DataFrame([{"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string.tolist())}])]
                                   , ignore_index=True)

    temp_mask_check = []
    mask_check = predict
    temp_mask_check.append(mask_check)
    oms_check = np.array(temp_mask_check)
    oms_check = oms_check.reshape([oms_check.shape[0], input_size*input_size]).astype(int)
    string_check = oms_check.flatten()

    submission_check = pd.concat([submission_check, pd.DataFrame([{"image_id" : file_name, "PredictionString" : ' '.join(str(e) for e in string_check.tolist())}])]
                                   , ignore_index=True)
                                   


submission_check.to_csv(os.path.join(check_dir, f'csv/submission_{epoch}.csv'), index=False)
submission.to_csv(os.path.join(cfg.work_dir, f'submission_{epoch}.csv'), index=False)

In [8]:
submission.head()

,image_id,PredictionString
0,batch_01_vt/0021.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
1,batch_01_vt/0028.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
2,batch_01_vt/0031.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
3,batch_01_vt/0032.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...
4,batch_01_vt/0076.jpg,0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 ...


In [63]:
len(submission_check['PredictionString'][1])

524287